In [ ]:
import random as rd
import pandas as pd

from cosapp.drivers import NonLinearSolver, RungeKutta, MonteCarlo, LinearDoE, RunSingleCase
from cosapp.utils.distributions import Normal
from cosapp.recorders import DataFrameRecorder
from cosapp_lab.widgets import SysExplorer

from cpu.systems import CPUSystem

In [ ]:
sampleNumbers=1001
datasetSize=1000
percentageBroken=20
rd.seed(9)

In [ ]:
cpu = CPUSystem("cpu")
cpu.fan.mass_flow_scalar=1.

design = cpu.add_driver(NonLinearSolver('solver'))
design.extend(cpu.design_methods["exchanger_surface"])

design.runner.set_values({"fan.T_air": 40., "T_cpu": 80., "cpu.usage": 100.})

# run design
cpu.run_drivers()

In [ ]:
cpu.exchanger.surface

In [ ]:
doe = cpu.add_driver(LinearDoE('doe'))

In [ ]:
time_driver = doe.add_child(RungeKutta(order=3, history = True))
solver = time_driver.add_child(NonLinearSolver('solver', max_iter=10, factor=1.0))
time_driver.time_interval = ([0, 30])
time_driver.dt = 0.5

# Define a simulation scenario
time_driver.set_scenario(
    init = {'T_cpu': 30,
           'fan.mass_flow_scalar':1.
           },
    values = {
        #"fan.T_air": 40., mettre en commentaire !!! sinon remet à zéro quand on appelle time_driver
        "cpu.usage": "100 if time <20 else 0."}
)

In [ ]:
doe.add_input_var({'fan.T_air': {"lower": 0., "upper": 30., "count": sampleNumbers}})

In [ ]:
doe.add_recorder(DataFrameRecorder(includes=['fan.tension', 'cpu.usage', 'T_cpu', 'fan.T_air', "exchanger.surface"]))

In [ ]:
cpu.fan.T_air

In [ ]:
cpu.run_drivers()

# Dataset creation

In [ ]:
df = doe.recorder.data

In [ ]:
working=[True for k in range(len(df))]

In [ ]:
df=df.assign(working=working)
df

In [ ]:
time_driver.set_scenario(
    init = {'T_cpu': 30,
           'fan.mass_flow_scalar':0.
           },
    values = {
        #"fan.T_air": 40., mettre en commentaire !!! sinon remet à zéro quand on appelle time_driver
        "cpu.usage": "100 if time <20 else 0."}
)

In [ ]:
cpu.run_drivers()

# Test set creation

In [ ]:
df2=doe.recorder.data

In [ ]:
working=[False for k in range(len(df2))]

In [ ]:
df2=df2.assign(working=working)
df2

In [ ]:
dfclean=df.drop(['Section', 'Status', 'Error code', 'Reference', 'cpu.usage', 'exchanger.surface'], axis=1)

In [ ]:
df2clean=df2.drop(['Section', 'Status', 'Error code', 'Reference', 'cpu.usage', 'exchanger.surface'], axis=1)

In [ ]:
dataset=[]
for k in range(datasetSize):
    if k < datasetSize*percentageBroken/100:
        i=rd.randint(0, len(df2clean)-1)
        dataset.append(df2clean.iloc[i])
        df2clean=df2clean.drop(df2clean.index[i])
    else:
        i=rd.randint(0, len(dfclean)-1)
        dataset.append(dfclean.iloc[i])
        dfclean=dfclean.drop(dfclean.index[i])
    dataset[k].name=k

dfclean=df.drop(['Section', 'Status', 'Error code', 'Reference', 'cpu.usage', 'exchanger.surface'], axis=1)
df2clean=df2.drop(['Section', 'Status', 'Error code', 'Reference', 'cpu.usage', 'exchanger.surface'], axis=1)

In [ ]:
"""
dataset=[]
for k in range(tailleDataset):
    if k < tailleDataset*percentageCasse/100:
        i=rd.randint(0, len(dfclean)-1)
        dataset.append(df2clean.iloc[i])
    else:
        i=rd.randint(0, len(dfclean)-1)
        dataset.append(dfclean.iloc[i])
    dataset[k].name=k
"""

In [ ]:
type(dfclean.iloc[1])

In [ ]:
cols=['T_cpu', 'fan.T_air', 'fan.tension', 'working']
dfFinal=pd.DataFrame(dataset, columns=cols)

In [ ]:
dfFinal

In [ ]:
dfFinal.to_csv(f"dataset_{datasetSize}_cases_{percentageBroken}_percent_broken.csv", index=False)

In [ ]:
dataset=[]
for k in range(datasetSize):
    dataset.append(df2clean.iloc[k])
    dataset.append(dfclean.iloc[k])
    dataset[2*k].name=2*k
    dataset[2*k+1].name=2*k+1

In [ ]:
len(dataset)

In [ ]:
cols=['T_cpu', 'fan.T_air', 'fan.tension', 'working']
testfinal=pd.DataFrame(dataset, columns=cols)
testfinal.to_csv(f"test_set_{datasetSize}_cases_{percentageBroken}_percent_broken.csv", index=False)